# Chat Completion API

In [1]:
!pip install openai

## 환경설정
1. OpenAI 사이트에서 API 키 발급
2. google colab - secret 탭에 `OPENAI_API_KEY` 를 등록해야 한다.

In [2]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [3]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

In [4]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "너는 아주 친절하고, 많은 도움을 주는 챗봇이야~"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "안녕~ 내 이름은 차은우야~"
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [6]:
print(response)

# ChatCompletion( choices=[ Choice( message=ChatCompletionMessage( content="여기가 실제 응답부분" ) ) ] )
print(response.choices[0].message.content)

ChatCompletion(id='chatcmpl-Bls5Vd6xi3mA4NjSn2EaKxpVUd7cr', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='안녕하세요, 차은우님! 반갑습니다. 어떻게 도와드릴까요?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1750749029, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_34a54ae93c', usage=CompletionUsage(completion_tokens=19, prompt_tokens=41, total_tokens=60, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
안녕하세요, 차은우님! 반갑습니다. 어떻게 도와드릴까요?


## 프롬프팅의 기본구성

https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/

프롬프팅에 꼭 들어가있어야 할 내용들

1. Instruction ( 지시사항 )
2. Context ( 문맥. 정확히 어떤걸 해야 하는지 나열 )
3. Input Data/Example ( 입력데이터나 예시 )
4. Output Indicator ( 출력 지시사항. 어떤 형식으로 출력할지 )

**`매우 중요함!!`**

## 예제 1 : 기사 제목 교정
- 기자들이 송고한 기사에서 제목을 추출하고, 표현 조정
- 프랑스 AFP 속보시스템에서 도입되어 사용한 적 있음.

In [7]:
title_before = "테이의 FM 개꿀 라디오 방송에 주목해주세요. 꿀잠 쌉가능. 아 얼른 재취하고싶다."

# LLM 의 역할, 페르소나, 지시사항을 꼼꼼히 작성
system_message = """
기자들이 송고한 제목에서 맞춤법, 문법, 의미, 어조, 그 밖에 기사에 어울리지 않는 것들을 교정해주세요.

- 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
- 독자의 관심을 끌 수 있도록 간결하고 임팩트있는 표현을 사용하세요.
- 어조가 지나치게 감정적이거나 부정적인 경우, 표현을 완화하거나 중립적인 어조로 수정하세요.
- 주제에 맞지 않는 문구는 삭제해주세요.
- 비속어가 포함되어있는 경우, 비속어를 반드시 제거하고 의미를 적절히 유지하도록 제목을 교정하세요.

# Step
1. 기사 제목을 읽고, 주요 내용을 이해하세요.
2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
3. 맞춤법, 문법, 의미 전달의 정확성을 점검하고 적절히 수정하세요.
4. 제목이 자연스럽고, 독자에게 매력적으로 어필할 수 있는지 점검하고 간결하게 정리하세요.

# Output Format
기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

- 원래 제목 : [기사의 원래 제목]
- 교정 제목 : [교정된 기사 제목]

# Examples
- 원래 제목 : "어제 서울에서 졸라 불이 크게 나서 수백명이 대피했고, 꼰대들이 가장 큰 피해자였다."
- 교정 제목 : "서울에서 대형화제로 수백명 대피, 중장년층 피해 커"

# Extra instructions
- 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버리면 안됩니다.
- 지역명, 시간 등의 중요정보는 명확하게 유지하세요.
- 제목이 특정 집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

"""

user_message = f"""
다음 기사 제목을 간결히 수정해주세요.

{title_before}
"""

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": system_message
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": user_message
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

print(response, "\n")
print(response.choices[0].message.content)

ChatCompletion(id='chatcmpl-BlsUczmenCTSnkhbxXRTcJKSlvOvw', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='- 원래 제목 : "테이의 FM 개꿀 라디오 방송에 주목해주세요. 꿀잠 쌉가능. 아 얼른 재취하고싶다."\n- 교정 제목 : "테이의 FM 라디오 방송, 편안한 수면을 위한 추천!"', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1750750586, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_34a54ae93c', usage=CompletionUsage(completion_tokens=61, prompt_tokens=473, total_tokens=534, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))) 

- 원래 제목 : "테이의 FM 개꿀 라디오 방송에 주목해주세요. 꿀잠 쌉가능. 아 얼른 재취하고싶다."
- 교정 제목 : "테이의 FM 라디오 방송, 편안한 수면을 위한 추천!"


In [8]:
# 함수 리팩토링
def audit_news_title(title, model="gpt-4o-mini", temperature=1.0, top_p=1.0):
  system_message = """
  기자들이 송고한 제목에서 맞춤법, 문법, 의미, 어조, 그 밖에 기사에 어울리지 않는 것들을 교정해주세요.

  - 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
  - 독자의 관심을 끌 수 있도록 간결하고 임팩트있는 표현을 사용하세요.
  - 어조가 지나치게 감정적이거나 부정적인 경우, 표현을 완화하거나 중립적인 어조로 수정하세요.
  - 주제에 맞지 않는 문구는 삭제해주세요.
  - 비속어가 포함되어있는 경우, 비속어를 반드시 제거하고 의미를 적절히 유지하도록 제목을 교정하세요.

  # Step
  1. 기사 제목을 읽고, 주요 내용을 이해하세요.
  2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
  3. 맞춤법, 문법, 의미 전달의 정확성을 점검하고 적절히 수정하세요.
  4. 제목이 자연스럽고, 독자에게 매력적으로 어필할 수 있는지 점검하고 간결하게 정리하세요.

  # Output Format
  기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

  - 원래 제목 : [기사의 원래 제목]
  - 교정 제목 : [교정된 기사 제목]

  # Examples
  - 원래 제목 : "어제 서울에서 졸라 불이 크게 나서 수백명이 대피했고, 꼰대들이 가장 큰 피해자였다."
  - 교정 제목 : "서울에서 대형화제로 수백명 대피, 중장년층 피해 커"

  # Extra instructions
  - 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버리면 안됩니다.
  - 지역명, 시간 등의 중요정보는 명확하게 유지하세요.
  - 제목이 특정 집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

  """

  user_message = f"""
  다음 기사 제목을 간결히 수정해주세요.

  {title}
  """

  response = client.chat.completions.create(
    model=model,
    messages=[
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text": system_message
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": user_message
          }
        ]
      }
    ],
    response_format={
      "type": "text"
    },
    temperature=temperature,
    max_completion_tokens=2048,
    top_p=top_p,
    frequency_penalty=0,
    presence_penalty=0
  )
  print(response, "\n")

  return response.choices[0].message.content


title_before = "주말동안 천둥번개를 동반한 미친 폭우가 예상됩니다. 다들 쓸데없이 돌아다니지 말고 집에 처박혀있으세요."
print( audit_news_title(title_before) )

ChatCompletion(id='chatcmpl-BlshwoNuAWB69NrqZ4cKQj8UjZr44', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='- 원래 제목 : "주말동안 천둥번개를 동반한 미친 폭우가 예상됩니다. 다들 쓸데없이 돌아다니지 말고 집에 처박혀있으세요."\n- 교정 제목 : "주말 천둥번개 동반 폭우 예상, 외출 자제 권장"', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1750751412, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_34a54ae93c', usage=CompletionUsage(completion_tokens=73, prompt_tokens=510, total_tokens=583, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))) 

- 원래 제목 : "주말동안 천둥번개를 동반한 미친 폭우가 예상됩니다. 다들 쓸데없이 돌아다니지 말고 집에 처박혀있으세요."
- 교정 제목 : "주말 천둥번개 동반 폭우 예상, 외출 자제 권장"


## 영문텍스트에서 anki 단어장 만들기 ( 과제 )